In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pyautogui
import pandas as pd

driver = webdriver.Chrome()

driver.get("https://www.google.com/maps/search/信義區+餐廳")

scroll_times = 50
for _ in range(scroll_times):
    pyautogui.press('pagedown')
    time.sleep(1)

"""
//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[3]/div/a
//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[5]/div/a
//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[7]/div/a

如果藉由XPATH定位點擊，需要取得每個店家的XPATH來循環操作
"""

# #用CSS selector
# store = driver.find_element(By.CSS_SELECTOR,"#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div:nth-child(3) > div > a") 
# store.click()
# 
# #用XPATH
# store = driver.find_element(By.XPATH,"//*[@id=\"QA0Szd\"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[3]/div/a") 
# store.click()
# 
# #用tag name 找餐廳資訊
# store = driver.find_element(By.TAG_NAME,'a') 
# store.click()

# 藉由專屬類名取得店家資料
store = driver.find_elements(By.CLASS_NAME,'hfpxzc')
# store.click()



# 店家店名爬取打包進list 
store_name_list = []
for ele in store:
    name = ele.get_attribute('aria-label')
    store_name_list.append(name)

# 店家詳細資訊連結打包進list
store_href_list = []
for href in store:
    link = href.get_attribute('href')
    store_href_list.append(link)
    

# 點擊店家資訊後再點擊返回鍵返回至搜索首頁
# back = driver.find_element(By.CSS_SELECTOR,'#omnibox-singlebox > div > div.fKm1Mb > button > span')
# back.click()


In [2]:
# 檢查爬取下的店家資料是否對應
print(len(store_name_list))
print(len(store_href_list))

120
120


In [3]:
# 先建好dataframe，將爬到店家名稱和資訊連結先存入
store_data = {'RESTAURANT':store_name_list,'LINK':store_href_list}

df_store_info = pd.DataFrame(store_data)
df_store_info

,RESTAURANT,LINK
0,合.shabu ∣ 信義區頂級鍋物,https://www.google.com/maps/place/%E5%90%88.sh...
1,Miacucina信義店,https://www.google.com/maps/place/Miacucina%E4...
2,JK STUDIO Modern Asia (原JK STUDIO 新義法料理),https://www.google.com/maps/place/JK+STUDIO+Mo...
3,非常泰,https://www.google.com/maps/place/%E9%9D%9E%E5...
4,JOYCE EAST,https://www.google.com/maps/place/JOYCE+EAST/d...
...,...,...
115,銀座杏子日式豬排-信義威秀店,https://www.google.com/maps/place/%E9%8A%80%E5...
116,M One Cafe A11館,https://www.google.com/maps/place/M+One+Cafe+A...
117,十二籃 粥火鍋 逸仙店,https://www.google.com/maps/place/%E5%8D%81%E4...
118,We&Me Cafe好好文化創意,https://www.google.com/maps/place/We%26Me+Cafe...


In [4]:
# 店家初步資訊導出為csv
df_store_info.to_excel(r"\專案實作\地區餐廳評論分析\店家資訊連結.xlsx",index=False)

## 構想

### 作法一 
先取得所有網頁上店家所屬XPATH
用XPATH點擊進入 依序取得評論、星級等等所需資訊返回至搜尋結果，然後循環操作直到所有資料打包輸出為csv
可能相對費時麻煩，實作邏輯性較複雜

### 作法二
藉由class name以及屬性取得店家名稱以及店家資訊連結 ps 搜索首頁中鎖定單一店家的網頁代碼中資訊能直接取得僅有名稱及一個連結
需要再藉由連結進入才能取得該店家更多資訊 包括評論、星級等等
所以先取得店家名稱以及連結放進一個data frame中
之後再設計開啟該dataframe中店家連結進入網頁然後爬取所有需要資料，循環操作直到爬完

